In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

In [ ]:
import os

Foursquare_ID = os.environ.get('CLIENT_ID')
Foursquare_Secret = os.environ.get('CLIENT_SECRET')

print(Foursquare_ID)
print(Foursquare_Secret)
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
address_sorrento = 'Sorrento, Italy'

geolocator = Nominatim(user_agent="foursquare_agent")
location_sorrento = geolocator.geocode(address_sorrento)
latitude_sorrento = location.latitude
longitude_sorrento = location.longitude
print(latitude_sorrento, longitude_sorrento)

In [ ]:
search_query_sorrento = '4bf58dd8d48988d110941735' # Italian restaurants in Sorrento search results using a Foursquare category code
radius = 1000
limit = 100
print(' .... OK!')

In [ ]:
# Formatting for url search

url_sorrento = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(search_query_sorrento, CLIENT_ID, CLIENT_SECRET, latitude_sorrento, longitude_sorrento, VERSION, radius, limit)
url_sorrento

In [ ]:
df_sorrento = requests.get(url_sorrento).json()
print('df_sorrento created')

In [ ]:
# assign relevant part of JSON to venues
venues_sorrento1 = df_sorrento['response']['venues']

# tranform venues into a dataframe
df_sorrento_italian = json_normalize(venues_sorrento1)
df_sorrento_italian.head()

In [ ]:
df_sorrento_italian.shape

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_sorrento_italian.columns if col.startswith('location.')] + ['id']
df_sorrento_IT_filtered = df_sorrento_italian.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_sorrento_IT_filtered['categories'] = df_sorrento_IT_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_sorrento_IT_filtered.columns = [column.split('.')[-1] for column in df_sorrento_IT_filtered.columns]

df_sorrento_IT_filtered.head()

In [ ]:
search_query_sorrento1 = '4d4b7105d754a06374d81259' # Food places in Sorrento search results using a Foursquare category code
radius = 1000
limit = 100
print(' .... OK!')

In [ ]:
# Formatting for url search

url_sorrento1 = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(search_query_sorrento1, CLIENT_ID, CLIENT_SECRET, latitude_sorrento, longitude_sorrento, VERSION, radius, limit)
url_sorrento1

In [ ]:
df_sorrento1 = requests.get(url_sorrento1).json()
print('df_sorrento1 created')

In [ ]:
# assign relevant part of JSON to venues
venues_sorrento2 = df_sorrento1['response']['venues']

# tranform venues into a dataframe
df_sorrento_food = json_normalize(venues_sorrento2)
df_sorrento_food.head()

In [ ]:
df_sorrento_food.shape

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_sorrento_food.columns if col.startswith('location.')] + ['id']
df_sorrento_food_filtered = df_sorrento_food.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_sorrento_food_filtered['categories'] = df_sorrento_food_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_sorrento_food_filtered.columns = [column.split('.')[-1] for column in df_sorrento_food_filtered.columns]

df_sorrento_food_filtered.head()

In [ ]:
df_sorrento_IT_filtered.to_csv('sorrentoITfiltered.csv', index=False)

In [ ]:
df_sorrento_food_filtered.to_csv('sorrentoFoodfiltered.csv', index=False)

In [ ]:
# Exported files 'sorrentoITfiltered and sorrentoFoodfiltered' to create usable database imported below

df_final_sorrento = pd.read_csv("SorrentoFoodPlaceRatings.csv")
df_final_sorrento.head()

In [ ]:
venues_map_sorrento_final = folium.Map(location=[latitude_sorrento, longitude_sorrento], zoom_start=15) # generate map centred around Sorrento, IT

# add a green circle marker to represent the center of Sorrento, IT
folium.features.CircleMarker(
    [latitude_sorrento, longitude_sorrento],
    radius=10,
    color='green',
    popup='Sorrento',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map_sorrento_final)

# add the food places of Sorrento as blue circle markers
for lat, lng, label in zip(df_final_sorrento.lat, df_final_sorrento.lng, df_final_sorrento.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_sorrento_final)

# display map
venues_map_sorrento_final

In [ ]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sorrento_final_cluster_map = folium.Map(location = [latitude_sorrento, longitude_sorrento], zoom_start = 15)

# instantiate a mark cluster object for the incidents in the dataframe
sorrento_grouping = plugins.MarkerCluster().add_to(sorrento_final_cluster_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_final_sorrento.lat, df_final_sorrento.lng, df_final_sorrento.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(sorrento_grouping)

# display map
sorrento_final_cluster_map

In [ ]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="foursquare_agent")
location_sf = geolocator.geocode(address)
latitude_sf = location_sf.latitude
longitude_sf = location_sf.longitude
print(latitude_sf, longitude_sf)

In [ ]:
search_sf_italian = '4bf58dd8d48988d110941735' # Foursquare category ID for Italian Restaurants
radius_sf = 20000
limit_sf = 100
print('Search completed!')

In [ ]:
url_sf_italian = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(search_sf_italian, CLIENT_ID, CLIENT_SECRET, latitude_sf, longitude_sf, VERSION, radius_sf, limit_sf)
url_sf_italian

In [ ]:
# Italian Restaurants search in San Francisco, CA

results_sf = requests.get(url_sf_italian).json()
print('San Francisco Results!')

In [ ]:
# assign relevant part of JSON to venues
venues_sf = results_sf['response']['venues']

# tranform venues into a dataframe
df_sanfran = json_normalize(venues_sf)
df_sanfran.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_sanfran.columns if col.startswith('location.')] + ['id']
df_sanfran_italian_filtered = df_sanfran.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_sanfran_italian_filtered['categories'] = df_sanfran_italian_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_sanfran_italian_filtered.columns = [column.split('.')[-1] for column in df_sanfran_italian_filtered.columns]
df_sanfran_italian_filtered.head()

In [ ]:
df_sanfran_italian_filtered.to_csv('SanFranItalianFiltered.csv', index=False)

In [ ]:
search_sf_seafood = '4bf58dd8d48988d1ce941735' # Foursquare category ID for Seafood Restaurants
radius_sf = 20000
limit_sf = 100
print('Search completed!')

In [ ]:
url_sf_seafood = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(search_sf_seafood, CLIENT_ID, CLIENT_SECRET, latitude_sf, longitude_sf, VERSION, radius_sf, limit_sf)
url_sf_seafood

In [ ]:
# Seafood Restaurants search in San Francisco, CA

results_sf_seafood = requests.get(url_sf_seafood).json()
print('San Francisco Seafood Results!')

In [ ]:
# assign relevant part of JSON to venues
venues_sf_seafood = results_sf_seafood['response']['venues']

# tranform venues into a dataframe
df_sanfran_seafood = json_normalize(venues_sf_seafood)
df_sanfran_seafood.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_sanfran_seafood.columns if col.startswith('location.')] + ['id']
df_sanfran_seafood_filtered = df_sanfran_seafood.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_sanfran_seafood_filtered['categories'] = df_sanfran_seafood_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_sanfran_seafood_filtered.columns = [column.split('.')[-1] for column in df_sanfran_seafood_filtered.columns]
df_sanfran_seafood_filtered.head()

In [ ]:
df_sanfran_seafood_filtered.to_csv('SanFranSeafoodFiltered.csv', index=False)

In [ ]:
search_sf_steakhouse = '4bf58dd8d48988d1cc941735' # Foursquare category ID for Steakhouse Restaurants
radius_sf = 20000
limit_sf = 100
print('Search completed!')

In [ ]:
url_sf_steakhouse = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(search_sf_steakhouse, CLIENT_ID, CLIENT_SECRET, latitude_sf, longitude_sf, VERSION, radius_sf, limit_sf)
url_sf_steakhouse

In [ ]:
# Steakhouse Restaurants search in San Francisco, CA

results_sf_steakhouse = requests.get(url_sf_steakhouse).json()
print('San Francisco Steakhouse Results!')

In [ ]:
# assign relevant part of JSON to venues
venues_sf_steakhouse = results_sf_steakhouse['response']['venues']

# tranform venues into a dataframe
df_sanfran_steakhouse = json_normalize(venues_sf_steakhouse)
df_sanfran_steakhouse.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_sanfran_steakhouse.columns if col.startswith('location.')] + ['id']
df_sanfran_steakhouse_filtered = df_sanfran_steakhouse.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_sanfran_steakhouse_filtered['categories'] = df_sanfran_steakhouse_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_sanfran_steakhouse_filtered.columns = [column.split('.')[-1] for column in df_sanfran_steakhouse_filtered.columns]
df_sanfran_steakhouse_filtered.head()

In [ ]:
df_sanfran_steakhouse_filtered.to_csv('SanFranSteakhouseFiltered.csv', index=False)

In [ ]:
# import combined files for San Francisco Italian, seafood, and steakhouse restaurants

df_sanfran_final = pd.read_csv("SanFranAllFoodPlaces05262020.csv")
df_sanfran_final.head()

In [ ]:
df_sanfran_final.shape

In [ ]:
# create San Francisco map and display it
sanfran_final_map = folium.Map(location=[latitude_sf, longitude_sf], zoom_start=12)

# display the map of San Francisco
sanfran_final_map

In [ ]:
# instantiate a feature group for the incidents in the dataframe
sf_food_final = folium.map.FeatureGroup()

# loop through the restaurants and add each to the feature group
for lat, lng, in zip(df_sanfran_final.Y, df_sanfran_final.X):
    sf_food_final.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_final_map.add_child(sf_food_final)

In [ ]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_final_map = folium.Map(location = [latitude_sf, longitude_sf], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
sanfran_food_cluster_final = plugins.MarkerCluster().add_to(sanfran_final_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_sanfran_final.Y, df_sanfran_final.X, df_sanfran_final.Type):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(sanfran_food_cluster_final)

# display map
sanfran_final_map